In [54]:
import numpy as np
import pandas as pd
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [55]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [56]:
# load data
rawData = pd.read_csv("../input/train.csv").values
rawSubmission = pd.read_csv("../input/test.csv").values.astype("float32")

In [57]:
np.random.shuffle(rawData)

training, test = rawData[:32000,:], rawData[32000:,:]

# extract labels and features 
rawTrainY = training[:,0].astype("int")
trainY = np_utils.to_categorical(rawTrainY)

rawTestY = test[:,0].astype("int")
testY = np_utils.to_categorical(rawTestY)

trainRawX = training[:,1:].astype("float32")
trainNormX = trainRawX / 255.0
trainX = trainNormX.reshape( (32000,28,28,1) )

testRawX = test[:,1:].astype("float32")
testNormX = testRawX / 255.0
testX = testNormX.reshape( (10000,28,28,1) )

num_classes = trainY.shape[1]

In [58]:
model = Sequential()

model.add(Convolution2D(32, 5, 5, input_shape=(28, 28, 1), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2))) #add
model.add(Dropout(0.2))

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [59]:
# Compile model
epochs = 5
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_16 (Convolution2D) (None, 28, 28, 32)    832         convolution2d_input_8[0][0]      
____________________________________________________________________________________________________
dropout_16 (Dropout)             (None, 28, 28, 32)    0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 28, 28, 32)    25632       dropout_16[0][0]                 
____________________________________________________________________________________________________
maxpooling2d_8 (MaxPooling2D)    (None, 14, 14, 32)    0           convolution2d_17[0][0]           
___________________________________________________________________________________________

In [60]:
#model.fit(trainX, trainY, validation_data=(testX, testY), nb_epoch=epochs, batch_size=32)
model.fit(trainX, trainY, validation_data=(testX, testY), nb_epoch=epochs, batch_size=32)

# Final evaluation of the model
#scores = model.evaluate(testX, testY, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 32000 samples, validate on 10000 samples
Epoch 1/5
32000/32000 [==============================] - 366s - loss: 0.3286 - acc: 0.8908 - val_loss: 0.0792 - val_acc: 0.9730
Epoch 2/5
32000/32000 [==============================] - 410s - loss: 0.0864 - acc: 0.9728 - val_loss: 0.0568 - val_acc: 0.9818
Epoch 3/5
32000/32000 [==============================] - 406s - loss: 0.0654 - acc: 0.9796 - val_loss: 0.0475 - val_acc: 0.9852
Epoch 4/5
32000/32000 [==============================] - 398s - loss: 0.0538 - acc: 0.9835 - val_loss: 0.0449 - val_acc: 0.9857
Epoch 5/5
32000/32000 [==============================] - 436s - loss: 0.0495 - acc: 0.9843 - val_loss: 0.0421 - val_acc: 0.9867
Accuracy: 98.67%


In [61]:
model.save('3l_32f_5x5.h5')